# Chapter 8. Adversarial Search
## 8.1. Main components of a board game

In [13]:
from __future__ import annotations
from typing import NewType, List
from abc import ABC, abstractmethod
from enum import Enum

__Implement game components__

In [14]:
Move = NewType('Move', int)

class Piece:
    @property
    def opposite(self) -> Piece:
        """
        Switch player
        """
        raise NotImplementedError("Should be implemented by subclasses.")

class Board(ABC):
    @property
    @abstractmethod
    def turn(self) -> Piece:
        ...
        
    @abstractmethod
    def move(self, location: Move) -> Board:
        ...
    
    @property
    @abstractmethod
    def legal_moves(self) -> List[Move]:
        ...
    
    @property
    @abstractmethod
    def is_win(self) -> bool:
        ...
    
    @property
    def is_draw(self) -> bool:
        return (not self.is_win) and (len(self.legal_moves) == 0)
    
    @abstractmethod
    def evaluate(self, player: Piece) -> float:
        ...

## 8.2. Tic-tac-toe
__Slot representation__

In [25]:
class TTTPiece(Piece, Enum):
    X = "X"
    O = "O"
    E = " "
    
    @property
    def opposite(self) -> TTTPiece:
        if self == TTTPiece.X:
            return TTTPiece.O
        elif self == TTTPiece.O:
            return TTTPiece.X
        else:
            return TTTPiece.E
    
    def __str__(self) -> str:
        return self.value

__Managing the state of the game__

In [26]:
class TTTBoard(Board):
    def __init__(self, position: List[TTTPiece] = [TTTPiece.E] * 9, 
                 turn: TTTPiece = TTTPiece.X) -> None:
        self.position: List[TTTPiece] = position
        self._turn: TTTPiece = turn
    
    @property
    def turn(self) -> Piece:
        return self._turn
    
    def move(self, location: Move) -> Board:
        temp_position: List[TTTPiece] = self.position.copy()
        temp_position[location] = self._turn
        return TTTBoard(temp_position, self._turn.opposite)
    
    @property
    def legal_moves(self) -> List[Move]:
        return [Move(l) for l in range(len(self.position)) if self.position[l] == TTTPiece.E]
    
    @property
    def is_win(self) -> bool:
        # Checking 3 rows, 3 column, 2 diagonals
        return (self.position[0] == self.position[1] and self.position[0] == self.position[2] and self.position[0] != TTTPiece.E or 
                self.position[3] == self.position[4] and self.position[3] == self.position[5] and self.position[3] != TTTPiece.E or
                self.position[6] == self.position[7] and self.position[6] == self.position[8] and self.position[6] != TTTPiece.E or
                self.position[0] == self.position[3] and self.position[0] == self.position[6] and self.position[0] != TTTPiece.E or
                self.position[1] == self.position[4] and self.position[1] == self.position[7] and self.position[1] != TTTPiece.E or
                self.position[2] == self.position[5] and self.position[2] == self.position[8] and self.position[2] != TTTPiece.E or
                self.position[0] == self.position[4] and self.position[0] == self.position[8] and self.position[0] != TTTPiece.E or
                self.position[2] == self.position[4] and self.position[2] == self.position[6] and self.position[2] != TTTPiece.E)
    
    def evaluate(self, player: Piece) -> float:
        if self.is_win and self.turn == player:
            return -1
        elif self.is_win and self.turn != player:
            return 1
        else:
            return 0
    
    def __repr__(self) -> str:
        return f"""{self.position[0]}|{self.position[1]}|{self.position[2]}
-----
{self.position[3]}|{self.position[4]}|{self.position[5]}
-----
{self.position[6]}|{self.position[7]}|{self.position[8]}"""

__MiniMax implementation__

In [27]:
# Finding the best of all possible results for the first player
def minimax(board: Board, maximizing: bool, original_player: Piece, max_depth: int = 8) -> float:
    # Base case - the final position is achieved of final depth is achieved
    if board.is_win or board.is_draw or max_depth == 0:
        return board.evaluate(original_player)
    
    # Recursive case - maximize your profit or minimize oponents profit
    if maximizing:
        # The lowest possible value
        best_eval: float = float("-inf")
        for move in board.legal_moves:
            result: float = minimax(board.move(move), False, original_player, max_depth - 1)
            best_eval = max(result, best_eval)
        return best_eval
    else: # minimizing
        worst_eval: float = float("inf")
        for move in board.legal_moves:
            result = minimax(board.move(move), True, original_player, max_depth - 1)
            worst_eval = min(result, worst_eval)
        return worst_eval

__Finding the best possible step from the current position looking max_depth moves__

In [28]:
def find_best_move(board: Board, max_depth: int = 8) -> Move:
    best_eval: float = float("-inf")
    best_move: Move = Move(-1)
    for move in board.legal_moves:
        result: float = minimax(board.move(move), False, board.turn, max_depth)
        if result > best_eval:
            best_eval = result
            best_move = move
    return best_move

__Create unit testing__

In [29]:
import unittest

class TTTMinimaxTestCase(unittest.TestCase):
    def test_easy_position(self):
        # Wining in a one step
        to_win_easy_position: List[TTTPiece] = [TTTPiece.X, TTTPiece.O, TTTPiece.X, 
                                                TTTPiece.X, TTTPiece.E, TTTPiece.O,
                                                TTTPiece.E, TTTPiece.E, TTTPiece.O]
        test_board1: TTTBoard = TTTBoard(to_win_easy_position, TTTPiece.X)
        answer1: Move = find_best_move(test_board1)
        self.assertEqual(answer1, 6)
    
    def test_block_position(self):
        # Do not let the opponent win
        to_block_position: List[TTTPiece] = [TTTPiece.X, TTTPiece.E, TTTPiece.E, 
                                             TTTPiece.E, TTTPiece.E, TTTPiece.O,
                                             TTTPiece.E, TTTPiece.X, TTTPiece.O]
        test_board2: TTTBoard = TTTBoard(to_block_position, TTTPiece.X)
        answer2: Move = find_best_move(test_board2)
        self.assertEqual(answer2, 2)
    
    def test_hard_position(self):
        # Find the best move to win in two turns
        to_win_hard_position: List[TTTPiece] = [TTTPiece.X, TTTPiece.E, TTTPiece.E, 
                                                TTTPiece.E, TTTPiece.E, TTTPiece.O,
                                                TTTPiece.O, TTTPiece.X, TTTPiece.E]
        test_board3: TTTBoard = TTTBoard(to_win_hard_position, TTTPiece.X)
        answer3: Move = find_best_move(test_board3)
        self.assertEqual(answer3, 1)

__Launch test__

In [30]:
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], exit=False)

...
----------------------------------------------------------------------
Ran 3 tests in 0.007s

OK


__Player vs Minimax Algorithm__

In [35]:
board: Board = TTTBoard()

def get_player_move() -> Move:
    player_move: Move = Move(-1)
    while player_move not in board.legal_moves:
        play: int = int(input("Enter a legal square (0-8): "))
        player_move = Move(play)
    return player_move

while True:
    human_move: Move = get_player_move()
    board = board.move(human_move)
    print(board)
    if board.is_win:
        print("Human wins!")
        break
    elif board.is_draw:
        print("Draw!")
        break
    computer_move: Move = find_best_move(board)
    print(f"Computer move is {computer_move}")
    board = board.move(computer_move)
    print(board)
    if board.is_win:
        print("Computer wins!")
        break
    elif board.is_draw:
        print("Draw!")
        break

Enter a legal square (0-8): 1
 |X| 
-----
 | | 
-----
 | | 
Computer move is 0
O|X| 
-----
 | | 
-----
 | | 
Enter a legal square (0-8): 4
O|X| 
-----
 |X| 
-----
 | | 
Computer move is 7
O|X| 
-----
 |X| 
-----
 |O| 
Enter a legal square (0-8): 2
O|X|X
-----
 |X| 
-----
 |O| 
Computer move is 6
O|X|X
-----
 |X| 
-----
O|O| 
Enter a legal square (0-8): 8
O|X|X
-----
 |X| 
-----
O|O|X
Computer move is 3
O|X|X
-----
O|X| 
-----
O|O|X
Computer wins!


__Algorithm vs Algorithm__

In [38]:
board: Board = TTTBoard()

while True:
    player1_move: Move = find_best_move(board)
    print(f"Player1 move is {player1_move}")
    board = board.move(player1_move)
    print(board)
    if board.is_win:
        print("Player1 wins!")
        break
    elif board.is_draw:
        print("Draw!")
        break
    input("Move on")
        
    player2_move: Move = find_best_move(board)
    print(f"Player2 move is {player2_move}")
    board = board.move(player2_move)
    print(board)
    if board.is_win:
        print("Computer wins!")
        break
    elif board.is_draw:
        print("Draw!")
        break
    input("Move on")

Player1 move is 0
X| | 
-----
 | | 
-----
 | | 
Move on
Player2 move is 4
X| | 
-----
 |O| 
-----
 | | 
Move on
Player1 move is 1
X|X| 
-----
 |O| 
-----
 | | 
Move on
Player2 move is 2
X|X|O
-----
 |O| 
-----
 | | 
Move on
Player1 move is 6
X|X|O
-----
 |O| 
-----
X| | 
Move on
Player2 move is 3
X|X|O
-----
O|O| 
-----
X| | 
Move on
Player1 move is 5
X|X|O
-----
O|O|X
-----
X| | 
Move on
Player2 move is 7
X|X|O
-----
O|O|X
-----
X|O| 
Move on
Player1 move is 8
X|X|O
-----
O|O|X
-----
X|O|X
Draw!


## 8.3. Connect four
__Connect Four game machinery__

In [39]:
from typing import List, Optional, Tuple
from enum import Enum

In [40]:
class C4Piece(Piece, Enum):
    B = "B"
    R = "R"
    E = " "
    
    @property
    def opposite(self) -> C4Piece:
        if self == C4Piece.B:
            return C4Piece.R
        elif self == C4Piece.R:
            return C4Piece.B
        else:
            return C4Piece.E
    
    def __str__(self) -> str:
        return self.value

__Creating all winning positions__

In [43]:
def generate_segments(num_columns: int, num_rows: int, segment_length: int) -> List[List[Tuple[int, int]]]:
    segments: List[List[Tuple[int, int]]] = []
    # Vertical winning segments
    for c in range(num_columns):
        for r in range(num_rows - segment_length + 1):
            segment: List[Tuple[int, int]] = []
            for t in range(segment_length):
                segment.append((c, r + t))
            segments.append(segment)
    
    # Horizontal winning segments
    for c in range(num_columns - segment_length + 1):
        for r in range(num_rows):
            segment: List[Tuple[int, int]] = []
            for t in range(segment_length):
                segment.append((c + t, r))
            segments.append(segment)
            
    # generate the bottom left to top right diagonal segments
    for c in range(num_columns - segment_length + 1):
        for r in range(num_rows - segment_length + 1):
            segment = []
            for t in range(segment_length):
                segment.append((c + t, r + t))
            segments.append(segment)

    # generate the top left to bottom right diagonal segments
    for c in range(num_columns - segment_length + 1):
        for r in range(segment_length - 1, num_rows):
            segment = []
            for t in range(segment_length):
                segment.append((c + t, r - t))
            segments.append(segment)
    return segments

__Creating board__

In [44]:
class C4Board(Board):
    NUM_ROWS: int = 6
    NUM_COLUMNS: int = 7
    SEGMENT_LENGTH: int = 4
    SEGMENTS: List[List[Tuple[int, int]]] = generate_segments(NUM_COLUMNS, NUM_ROWS, SEGMENT_LENGTH)
    
    class Column:
        def __init__(self) -> None:
            self._container: List[C4Piece] = []
        
        @property
        def full(self) -> bool:
            return len(self._container) == C4Board.NUM_ROWS
        
        def push(self, item: C4Piece) -> None:
            if self.full:
                raise OverflowError("Trying to push piece to full collumn")
            self._container.append(item)
        
        def __getitem__(self, index: int) -> C4Piece:
            if index > len(self._container) - 1:
                return C4Piece.E
            return self._container[index]
        
        def __repr__(self) -> str:
            return repr(self._container)
        
        def copy(self) -> C4Board.Column:
            temp: C4Board.Column = C4Board.Column()
            temp._container = self._container.copy()
            return temp
    
    def __init__(self, position: Optional[List[C4Board.Column]] = None, turn: C4Piece = C4Piece.B) -> None:
        if position is None:
            self.position: List[C4Board.Column] = [C4Board.Column() for _ in range(C4Board.NUM_COLUMNS)]
        else:
            self.position = position
        self._turn: C4Piece = turn
            
    @property
    def turn(self) -> C4Piece:
        return self._turn
    
    def move(self, location: Move) -> Board:
        temp_position: List[C4Board.Column] = self.position.copy()
        for c in range(C4Board.NUM_COLUMNS):
            temp_position[c] = self.position[c].copy()
        temp_position[location].push(self._turn)
        return C4Board(temp_position, self._turn.opposite)
    
    @property
    def legal_moves(self) -> List[Move]:
        return [Move(c) for c in range(C4Board.NUM_COLUMNS) if not self.position[c].full]
    
    def _count_segment(self, segment: List[Tuple[int, int]]) -> Tuple[int, int]:
        black_count: int = 0
        red_count: int = 0
        for column, row in segment:
            if self.position[column][row] == C4Piece.B:
                black_count += 1
            elif self.position[column][row] == C4Piece.R:
                red_count += 1
        return black_count, red_count
    
    @property
    def is_win(self) -> bool:
        for segment in C4Board.SEGMENTS:
            black_count, red_count = self._count_segment(segment)
            if black_count == 4 or red_count == 4:
                return True
        return False
    
    def _evaluate_segment(self, segment: List[Tuple[int, int]], player: Piece) -> float:
        black_count, red_count = self._count_segment(segment)
        if red_count > 0 and black_count > 0:
            return 0 # Mixed segments are neutral
        count: int = max(red_count, black_count)
        score: float = 0
        if count == 2:
            score = 1
        elif count == 3:
            score = 100
        elif count == 4:
            score = 1000000
        color: C4Piece = C4Piece.B
        if red_count > black_count:
            color = C4Piece.R
        if color != player:
            return -score
        return score
    
    def evaluate(self, player: Piece) -> float:
        total: float = 0
        for segment in C4Board.SEGMENTS:
            total += self._evaluate_segment(segment, player)
        return total
    
    def __repr__(self) -> str:
        display: str = ""
        for r in reversed(range(C4Board.NUM_ROWS)):
            display += "|"
            for c in range(C4Board.NUM_COLUMNS):
                display += f"{self.position[c][r]}" + "|"
            display += "\n"
        return display

__Prepare for launching the game__

In [45]:
board: Board = C4Board()
    
def get_player_move() -> Move:
    player_move: Move = Move(-1)
    while player_move not in board.legal_moves:
        play: int = int(input("Enter a legal column (0-6): "))
        player_move = Move(play)
    return player_move

__Main game cycle__

In [46]:
while True:
    human_move: Move = get_player_move()
    board = board.move(human_move)
    print(board)
    if board.is_win:
        print("Human wins!")
        break
    elif board.is_draw:
        print("Draw!")
        break
    
    computer_move: Move = find_best_move(board, 3)
    print(f"Computer move is {computer_move}")
    board = board.move(computer_move)
    print(board)
    if board.is_win:
        print("Computer_wins!")
        break
    elif board.is_draw:
        print("Draw!")
        break

Enter a legal column (0-6): 2
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | |B| | | | |

Computer move is 0
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
|R| |B| | | | |

Enter a legal column (0-6): 2
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | |B| | | | |
|R| |B| | | | |

Computer move is 2
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | |R| | | | |
| | |B| | | | |
|R| |B| | | | |

Enter a legal column (0-6): 1
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | |R| | | | |
| | |B| | | | |
|R|B|B| | | | |

Computer move is 1
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | |R| | | | |
| |R|B| | | | |
|R|B|B| | | | |

Enter a legal column (0-6): 3
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | |R| | | | |
| |R|B| | | | |
|R|B|B|B| | | |

Computer move is 4
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | |R| | | | |
| |R|B| | | | |
|R|B|B|B|R| | |

Enter a legal column (0-6): 

__Improving Minimax with alpha-beta pruning__

In [54]:
def alphabeta(board: Board, maximizing: bool, original_player: Piece, max_depth: int = 8, alpha: float = float("-inf"), beta: float = float("inf")) -> float:
    # Base case – terminal position or maximum depth reached
    if board.is_win or board.is_draw or max_depth == 0:
        return board.evaluate(original_player)

    # Recursive case - maximize your gains or minimize the opponent's gains
    if maximizing:
        for move in board.legal_moves:
            result: float = alphabeta(board.move(move), False, original_player, max_depth - 1, alpha, beta)
            alpha = max(result, alpha)
            if beta <= alpha:
                break
        return alpha
    else:  # minimizing
        for move in board.legal_moves:
            result = alphabeta(board.move(move), True, original_player, max_depth - 1, alpha, beta)
            beta = min(result, beta)
            if beta <= alpha:
                break
        return beta


# Find the best possible move in the current position
# looking up to max_depth ahead
def find_best_move(board: Board, max_depth: int = 8) -> Move:
    best_eval: float = float("-inf")
    best_move: Move = Move(-1)
    for move in board.legal_moves:
        result: float = alphabeta(board.move(move), False, board.turn, max_depth)
        if result > best_eval:
            best_eval = result
            best_move = move
    return best_move

__Algorithm vs algorithm__

In [55]:
board: Board = C4Board()

while True:
    player1_move: Move = find_best_move(board, 3)
    print(f"Player1 move is {player1_move}")
    board = board.move(player1_move)
    print(board)
    if board.is_win:
        print("Player1 wins!")
        break
    elif board.is_draw:
        print("Draw!")
        break
    input()
    
    player2_move: Move = find_best_move(board, 5)
    print(f"Player2 move is {player2_move}")
    board = board.move(player2_move)
    print(board)
    if board.is_win:
        print("Player2 wins!")
        break
    elif board.is_draw:
        print("Draw!")
        break
    input()

Player1 move is 2
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | |B| | | | |


Player2 move is 2
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | |R| | | | |
| | |B| | | | |


Player1 move is 1
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | |R| | | | |
| |B|B| | | | |


Player2 move is 3
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | |R| | | | |
| |B|B|R| | | |


Player1 move is 3
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | |R|B| | | |
| |B|B|R| | | |


Player2 move is 2
| | | | | | | |
| | | | | | | |
| | | | | | | |
| | |R| | | | |
| | |R|B| | | |
| |B|B|R| | | |


Player1 move is 2
| | | | | | | |
| | | | | | | |
| | |B| | | | |
| | |R| | | | |
| | |R|B| | | |
| |B|B|R| | | |


Player2 move is 2
| | | | | | | |
| | |R| | | | |
| | |B| | | | |
| | |R| | | | |
| | |R|B| | | |
| |B|B|R| | | |


Player1 move is 0
| | | | | | | |
| | |R| | | | |
| | |B| | | | |
| | |R